In [1]:
import numpy as np
import numba as nb

In [2]:
# 晶格相关参数
L = 4

#强化学习相关参数
c = 1       #cost
alpha = 0.1 #学习率
epsilon = 0.4 #随机参数

In [3]:
S = np.ones([L, L])
Q = np.ones([L, L, 2, 2])  #位置、位置、状态（与大多数状态相同为1,否则为0）、行动（改变为1,不变为0）

In [4]:
# 一次更新过程
for _ in range(L**2):
    x = np.random.choice(L)
    y = np.random.choice(L)

    # 判断属于大多数or少数
    near = S[(x+1)%L, y]+S[(x-1)%L, y]+S[x, (y+1)%L]+S[x, (y-1)%L]+0.1 #将0也划分为大多数
    stat = int((np.sign(S[x, y]*near)+1)/2)

    # 选择是否翻转
    action = 1
    if np.random.random() < epsilon :
        action = np.random.choice([1, 0])
        S[x, y] = S[x, y] * (action-0.5)*2
    else:
        if Q[x, y, stat, 1] > Q[x, y, stat, 0]:
            S[x, y] = S[x, y] * -1
            action = 1
        else: action = 0

    # 更新
    if int((np.sign(S[x, y]*near)+1)/2) == 1 : c = 0
    else: c = 1

    Q[x, y, stat, action] = Q[x, y, stat, action] - alpha*(c-Q[x, y, stat, action])

print(Q)

[[[[1.    1.   ]
   [1.    1.   ]]

  [[1.    1.   ]
   [1.    1.1  ]]

  [[1.    1.   ]
   [1.    1.   ]]

  [[1.    1.   ]
   [1.    1.   ]]]


 [[[1.    1.   ]
   [1.231 1.   ]]

  [[1.    1.   ]
   [1.    1.   ]]

  [[1.    1.   ]
   [1.    1.   ]]

  [[1.    1.   ]
   [1.    1.   ]]]


 [[[1.    1.   ]
   [1.21  1.   ]]

  [[1.    1.   ]
   [1.    1.1  ]]

  [[1.    1.   ]
   [1.    1.   ]]

  [[1.    1.   ]
   [1.11  1.   ]]]


 [[[1.    1.   ]
   [1.    1.   ]]

  [[1.    1.   ]
   [1.    1.   ]]

  [[1.    1.   ]
   [1.    1.1  ]]

  [[1.    1.   ]
   [1.1   1.   ]]]]


In [5]:
print(int((np.sign(3.2)+1)/2))

1


In [6]:
print(np.random.choice([1, -1]))

-1
